# County level yield analysis: Figure 3

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as ss
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity
import matplotlib.pyplot as plt
from matplotlib import gridspec

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Avenir'
plt.rcParams['mathtext.rm'] = 'Avenir'
plt.rcParams['mathtext.it'] = 'Avenir:italic'
plt.rcParams['mathtext.bf'] = 'Avenir:bold'
plt.rcParams['font.size'] = 22
plt.rcParams['axes.edgecolor']='#333F4B'
plt.rcParams['axes.linewidth']=0.8
plt.rcParams['xtick.color']='#333F4B'
plt.rcParams['ytick.color']='#333F4B'
plt.rcParams['pdf.fonttype'] = 42

In [3]:
# Get both ensembles
from combine import combine_nex, combine_cmip
nex_all = combine_nex()
cmip_all = combine_cmip()

In [5]:
# Set indexing
nex_all.set_index(['GEOID','Year'], inplace=True)
cmip_all.set_index(['GEOID','Year'], inplace=True)

### Fit bandwidths for each county

In [25]:
# Interquartile range
def get_iqr(dat):
    return np.quantile(dat, 0.75) - np.quantile(dat, 0.25)

# Silverman bandwidth estimate
def silverman(dat):
    return 0.9 * np.min([np.std(dat), get_iqr(dat)/1.34]) * len(dat)**-0.2

In [44]:
%%time
#############################
#### WARNING: this took 1hr 20mins to run on my local machine. Importing the results (next cell) will be much faster
############################
NEXres = {}
CMIPres = {}
for geoid in nex_all.index.unique(level = 'GEOID'):
    # Ensembles
    NEXens = nex_all.loc[geoid].drop(columns=['GMFD']).to_numpy().flatten()
    CMIPens = cmip_all.loc[geoid].drop(columns=['GMFD']).to_numpy().flatten()
    # Obs
    GMFDobs = nex_all.loc[geoid].filter(['GMFD']).to_numpy()
    # Find best bandswidth
    nexgrid = GridSearchCV(KernelDensity(), {'bandwidth': np.linspace(0.5*silverman(NEXens), 2*silverman(NEXens), 20)})
    cmipgrid = GridSearchCV(KernelDensity(), {'bandwidth': np.linspace(0.5*silverman(CMIPens), 2*silverman(CMIPens), 20)})
    nexgrid.fit(NEXens[:, None])
    cmipgrid.fit(CMIPens[:, None])
    NEXres.update({geoid : nexgrid.best_params_['bandwidth']})
    CMIPres.update({geoid : cmipgrid.best_params_['bandwidth']})

# Store results
NEX_res = pd.DataFrame.from_dict(NEXres, orient = 'index', columns = ['bw'])
NEX_res.index.name = 'GEOID'
NEX_res.to_csv('./bandwidths/nex_yield_bws.csv')

CMIP_res = pd.DataFrame.from_dict(CMIPres, orient = 'index', columns = ['bw'])
CMIP_res.index.name = 'GEOID'
CMIP_res.to_csv('./bandwidths/cmip_yield_bws.csv')

CPU times: user 1h 20min 27s, sys: 25.7 s, total: 1h 20min 53s
Wall time: 1h 24min 14s


In [89]:
# Import results
NEX_res = pd.read_csv('./bandwidths/nex_yield_bws.csv')
NEX_res['GEOID'] = NEX_res['GEOID'].astype(str).str.zfill(5)
NEX_res.set_index(['GEOID'], inplace=True)
CMIP_res = pd.read_csv('./bandwidths/cmip_yield_bws.csv')
CMIP_res['GEOID'] = CMIP_res['GEOID'].astype(str).str.zfill(5)
CMIP_res.set_index(['GEOID'], inplace=True)

### Calculate tail probabilities for each test statistic, for each county

In [ ]:
%%time
#############################
#### WARNING: this took 7hrs to run on my local machine. Importing the results (next cell) will be much faster
############################
# Results dict
NEXtail = []
CMIPtail = []
# Loop over each county
for geoid in nex_all.index.unique(level='GEOID'):
    # Ensembles
    NEXens = nex_all.loc[geoid].drop(columns=['GMFD']).to_numpy().flatten()
    CMIPens = cmip_all.loc[geoid].drop(columns=['GMFD']).to_numpy().flatten()
    # Obs
    GMFDobs = nex_all.loc[geoid].filter(['GMFD']).to_numpy()
    # KDEs
    nex_kde_skl = KernelDensity(bandwidth=NEX_res.loc[geoid]['bw'])
    cmip_kde_skl = KernelDensity(bandwidth=CMIP_res.loc[geoid]['bw'])
    nex_kde_skl.fit(NEXens[:, None])
    cmip_kde_skl.fit(CMIPens[:, None])
    # Construct sample
    nex_sample = np.array([nex_kde_skl.sample(n_samples=len(GMFDobs)).flatten() for i in range(10000)])
    cmip_sample = np.array([cmip_kde_skl.sample(n_samples=len(GMFDobs)).flatten() for i in range(10000)])

    # SD
    nex_sd_sample = np.array([np.std(dat) for dat in nex_sample])
    cmip_sd_sample = np.array([np.std(dat) for dat in cmip_sample])
    nex_sd_ecdf = ECDF(nex_sd_sample)
    cmip_sd_ecdf = ECDF(cmip_sd_sample)
    gmfd_sd = np.std(GMFDobs)

    # MAD
    nex_mad_sample = np.array([ss.median_absolute_deviation(dat) for dat in nex_sample])
    cmip_mad_sample = np.array([ss.median_absolute_deviation(dat) for dat in cmip_sample])
    nex_mad_ecdf = ECDF(nex_mad_sample)
    cmip_mad_ecdf = ECDF(cmip_mad_sample)
    gmfd_mad = ss.median_absolute_deviation(GMFDobs)
    
    # Min
    nex_min_sample = np.array([dat.min() for dat in nex_sample])
    cmip_min_sample = np.array([dat.min() for dat in cmip_sample])
    nex_min_ecdf = ECDF(nex_min_sample)
    cmip_min_ecdf = ECDF(cmip_min_sample)
    gmfd_min = GMFDobs.min()
    
    # Store
    NEXtail.append({'GEOID': geoid,
                    'gmfd_sd': 1.-nex_sd_ecdf(gmfd_sd),
                    'gmfd_mad': 1.-nex_mad_ecdf(gmfd_mad)[0],
                    'gmfd_min': 1.-nex_min_ecdf(gmfd_min)
                   })
    CMIPtail.append({'GEOID': geoid,
                     'gmfd_sd': 1.-cmip_sd_ecdf(gmfd_sd),
                     'gmfd_mad': 1.-cmip_mad_ecdf(gmfd_mad)[0],
                     'gmfd_min': 1.-cmip_min_ecdf(gmfd_min)
                    })
    
# Store results
NEX_tail = pd.DataFrame.from_dict(NEXtail)
NEX_tail.to_csv('./output/nex_yield_pvals_60-05.csv', index=False)

CMIP_tail = pd.DataFrame.from_dict(CMIPtail)
CMIP_tail.to_csv('./output/cmip_yield_pvals_60-05.csv', index=False)